<a href="https://colab.research.google.com/github/tiadoherty/tia_doherty_code_sample/blob/main/CodeSample_Tia_Doherty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
import requests
import json
import numpy as np
from datetime import datetime, timedelta

In [2]:
#the original version of this script that I created pulls from an internal API at Comcast every week, for the purposes of this exercise I have created a
#simplified dataset for April and May that holds the columns needed for the transformations in the script. All data has been deidentified and I swapped
#out all original ticket ids, usernames, and timestamps with dummy entries generated by Python.
#logic demonstrated here: using loops, appending data, reading sets of data in a non-default file format like json

#access json files from github:
april_url = 'https://raw.githubusercontent.com/tiadoherty/tia_doherty_code_sample/main/april_tickets.json'
may_url = 'https://raw.githubusercontent.com/tiadoherty/tia_doherty_code_sample/main/may_tickets.json'

response_april = requests.get(april_url)
response_may = requests.get(may_url)

april_data = []
may_data = []

#create file-like object and loop through each json object, appending into the april and may lists:
with io.StringIO(response_april.text) as file:
    for line in file:
        april_data.append(pd.read_json(io.StringIO(line), typ='series'))

with io.StringIO(response_may.text) as file:
    for line in file:
        may_data.append(pd.read_json(io.StringIO(line), typ='series'))

#convert into pandas dataframe:
df_april = pd.DataFrame(april_data)
df_may = pd.DataFrame(may_data)

In [3]:
#quick validation on the two dataframes:
df_april.head()

,ticketId,createDate,problemSummary,status,accountNumber,region,escalated,frt,assigned_to,resolutionDetails,endtime
0,KYR361R47B,2024-04-02 18:15:23,Late Payment Fee Dispute,CLOSED,282416533844,NORTHEAST,True,2024-04-02 22:15:23,test200,None,2024-04-03 02:15:23
1,BA9TJJB3ZA,2024-04-04 05:47:41,Unreturned Equipment,CLOSED,11697715190,WEST,False,2024-04-04 06:47:41,test200,offered credit,2024-04-04 10:47:41
2,XM2O2FWQK6,2024-04-15 06:01:13,Duplicate Charge,CLOSED,311765306940,EAST,False,2024-04-15 11:01:13,test200,offered credit,2024-04-15 14:01:13
3,IQNAQCWW6R,2024-04-11 03:34:44,Billing Error,CLOSED,774886296613,EAST,True,2024-04-11 04:34:44,test200,None,2024-04-11 07:34:44
4,O08WYAIAOA,2024-04-09 18:54:08,Incorrect Package Pricing,CLOSED,994259136122,SOUTH,False,2024-04-09 19:54:08,test200,offered credit,2024-04-09 20:54:08


In [4]:
df_may.head()

,ticketId,createDate,problemSummary,status,accountNumber,region,escalated,frt,assigned_to,resolutionDetails,endtime
0,E2TTXNSO5J,2024-05-17 9:23:00,Unreturned Equipment,CLOSED,166061588498,EAST,True,2024-05-17 14:23:00,test200,offered credit,2024-05-17 18:23:00
1,L3O8GT1K2B,2024-05-28 1:01:35,Incorrect Package Pricing,CLOSED,627006274076,EAST,False,2024-05-28 2:01:35,test200,None,2024-05-28 6:01:35
2,22KRXVFXRF,2024-05-16 16:19:33,Promotion Not Applied,CLOSED,738994699288,SOUTH,False,2024-05-16 18:19:33,test200,offered credit,2024-05-16 20:19:33
3,7TFIQGFVQM,2024-05-29 17:21:13,Billing Error,CLOSED,777001096724,EAST,True,2024-05-29 18:21:13,test200,offered credit,2024-05-29 19:21:13
4,XKSJEMZESO,2024-05-10 6:28:48,Late Payment Fee Dispute,CLOSED,567062102783,WEST,False,2024-05-10 11:28:48,test200,None,2024-05-10 12:28:48


In [5]:
df_april.columns

Index(['ticketId', 'createDate', 'problemSummary', 'status', 'accountNumber',
       'region', 'escalated', 'frt', 'assigned_to', 'resolutionDetails',
       'endtime'],
      dtype='object')

In [6]:
df_may.columns

Index(['ticketId', 'createDate', 'problemSummary', 'status', 'accountNumber',
       'region', 'escalated', 'frt', 'assigned_to', 'resolutionDetails',
       'endtime'],
      dtype='object')

In [7]:
len(df_april)

45

In [8]:
len(df_may)

45

In [9]:
#merge these two dataframes to do analysis on both april and may data - using concat instead of merge to combine vertically

df_april.shape #output of this: (45,11)
df_may.shape #output of this: (45,11)
pd.concat([df_april,df_may]).shape #expecting: (90,11)

(90, 11)

In [10]:
df_merge=pd.concat([df_april,df_may])

In [11]:
#validating that the merged df length is 90:
len(df_merge)

90

In [12]:
#validating that the max date in the merged df is in May:
df_merge['createDate'].max()

'2024-05-30 4:24:10'

In [13]:
#validating that the min date in the merged df is in April:
df_merge['createDate'].min()

'2024-04-02 04:56:00'

In [14]:
#users requested that the final table have timestamp datatypes so need to cast the date columns to timestamp, the schema when pulling from APIs
#usually does not auto interpret timestamps

print(df_merge.dtypes)

ticketId             object
createDate           object
problemSummary       object
status               object
accountNumber         int64
region               object
escalated              bool
frt                  object
assigned_to          object
resolutionDetails    object
endtime              object
dtype: object


In [15]:
df_merge['createDate'] = pd.to_datetime(df_merge['createDate'])
df_merge['frt'] = pd.to_datetime(df_merge['frt'])
df_merge['endtime'] = pd.to_datetime(df_merge['endtime'])

In [16]:
print(df_merge.dtypes)

ticketId                     object
createDate           datetime64[ns]
problemSummary               object
status                       object
accountNumber                 int64
region                       object
escalated                      bool
frt                  datetime64[ns]
assigned_to                  object
resolutionDetails            object
endtime              datetime64[ns]
dtype: object


In [17]:
#for the ETL framework used at my job, there needs to be a column called incremental with a particular MM-DD-YYYY format which is
#used for daily journaling processes and defines the partitions in the table to make it faster to query. So here I am adding a new column for internal purposes

df_merge['incremental'] = df_merge['createDate'].dt.strftime('%m-%d-%Y')

In [18]:
df_merge[['ticketId', 'createDate','incremental']].head()

,ticketId,createDate,incremental
0,KYR361R47B,2024-04-02 18:15:23,04-02-2024
1,BA9TJJB3ZA,2024-04-04 05:47:41,04-04-2024
2,XM2O2FWQK6,2024-04-15 06:01:13,04-15-2024
3,IQNAQCWW6R,2024-04-11 03:34:44,04-11-2024
4,O08WYAIAOA,2024-04-09 18:54:08,04-09-2024


In [ ]:
#the reporting team wanted to be able to look at duration of customer service interactions at retail stores,
#so need to add a column that contains duration based on frt (first response time) and endtime in minutes

#chose one sample record with one ticketId to use for validation:
result = df_merge[df_merge['ticketId'] == 'BHQFJCPQUZ'][['ticketId', 'frt', 'endtime']]
print(result)

      ticketId                 frt             endtime
35  BHQFJCPQUZ 2024-05-05 01:34:32 2024-05-05 03:34:32


In [ ]:
df_merge['duration'] = (df_merge['endtime'] - df_merge['frt'])

In [ ]:
#checking that the duration column from the sample record above has been added and is calculating the correct result
result = df_merge[df_merge['ticketId'] == 'BHQFJCPQUZ'][['ticketId', 'frt', 'endtime','duration']]
display(result)

,ticketId,frt,endtime,duration
35,BHQFJCPQUZ,2024-05-05 01:34:32,2024-05-05 03:34:32,0 days 02:00:00


In [ ]:
#now updating the duration column to contain the duration length in minutes
df_merge['duration'] = df_merge['duration'].dt.total_seconds() / 60

result = df_merge[df_merge['ticketId'] == 'BHQFJCPQUZ'][['ticketId', 'frt', 'endtime','duration']]
display(result)

,ticketId,frt,endtime,duration
35,BHQFJCPQUZ,2024-05-05 01:34:32,2024-05-05 03:34:32,120.0


In [ ]:
#validating that the duration column looks correct for more sample records:
df_merge.head()

,ticketId,createDate,problemSummary,status,accountNumber,region,escalated,frt,assigned_to,resolutionDetails,endtime,incremental,duration
0,KYR361R47B,2024-04-02 18:15:23-04:00,Late Payment Fee Dispute,CLOSED,282416533844,NORTHEAST,True,2024-04-02 22:15:23,test200,None,2024-04-03 02:15:23,04-02-2024,240.0
1,BA9TJJB3ZA,2024-04-04 05:47:41-04:00,Unreturned Equipment,CLOSED,11697715190,WEST,False,2024-04-04 06:47:41,test200,offered credit,2024-04-04 10:47:41,04-04-2024,240.0
2,XM2O2FWQK6,2024-04-15 06:01:13-04:00,Duplicate Charge,CLOSED,311765306940,EAST,False,2024-04-15 11:01:13,test200,offered credit,2024-04-15 14:01:13,04-15-2024,180.0
3,IQNAQCWW6R,2024-04-11 03:34:44-04:00,Billing Error,CLOSED,774886296613,EAST,True,2024-04-11 04:34:44,test200,None,2024-04-11 07:34:44,04-11-2024,180.0
4,O08WYAIAOA,2024-04-09 18:54:08-04:00,Incorrect Package Pricing,CLOSED,994259136122,SOUTH,False,2024-04-09 19:54:08,test200,offered credit,2024-04-09 20:54:08,04-09-2024,60.0


In [ ]:
#also to demonstrate conditional logic, have added a new column that flags any records where the customer service interaction
#involved a billing error and the interaction was escalated to a manager
df_merge['billing_escalations'] = np.where(
    (df_merge['problemSummary'] == 'Billing Error') & (df_merge['escalated'] == True), 1, 0
)

In [ ]:
#validating that there are 1s and 0s added in the new column:
print(df_merge['billing_escalations'].value_counts())

billing_escalations
0    83
1     7
Name: count, dtype: int64


In [ ]:
# second validation by filtering rows where the conditions should have resulted in a 1:
print(df_merge[(df_merge['problemSummary'] == 'Billing Error') & (df_merge['escalated'] == True)][['problemSummary', 'escalated', 'billing_escalations']])

   problemSummary  escalated  billing_escalations
3   Billing Error       True                    1
34  Billing Error       True                    1
36  Billing Error       True                    1
3   Billing Error       True                    1
12  Billing Error       True                    1
37  Billing Error       True                    1
39  Billing Error       True                    1


In [ ]:
#normally use sql from the final table in s3 for these types of details, but for this exercise I am showing
#how I would use pandas to get summary stats of different interactions in April and May
duration_frequencies = df_merge['duration'].value_counts()

duration_stats = df_merge['duration'].describe()

total_billing_escalations = df_merge['billing_escalations'].sum()

In [ ]:
print(duration_frequencies)

duration
180.0    28
240.0    26
60.0     22
120.0    14
Name: count, dtype: int64


In [ ]:
print(duration_stats)

count     90.000000
mean     158.666667
std       68.682628
min       60.000000
25%      120.000000
50%      180.000000
75%      240.000000
max      240.000000
Name: duration, dtype: float64


In [ ]:
print(total_billing_escalations)

7


In [ ]:
#in the script I wrote for my job this df would be written into our s3 datalake as a parquet file because we use Spark,
#and then the table is used by BI for reporting. For this exercise, you can run this cell and it will automatically download to your device
from google.colab import files
df_merge.to_csv('final_april_may_df.csv')
files.download('final_april_may_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>